In [1]:
%%capture
!pip install germansentiment

In [1]:
!pip install openpyxl

  Using cached openpyxl-3.1.0-py2.py3-none-any.whl (250 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


# Sentiment Approach with Swear word lexicon combined

In [2]:
import pandas as pd
from ipywidgets import IntProgress
from IPython.display import display
import time

In [8]:
# read in all tweets with swear words
# The dataset was created in the "schimpf.ipynb" Notebook
swear_df = pd.read_csv("swear_word_tweets.csv")
swear_df

,text
0,1/2\nIhr #Verräter\nIhr #Eidbrecher\nIhr #Verb...
1,@SWagenknecht Weil Sie das ja auch so gut wiss...
2,@ABW67484335 @susi_lins @SWagenknecht Ach wie ...
3,"@McI0vin @Bauernopfer1 @voglerk Ich weiß ja, i..."
4,@SevimDagdelen Dağdelen erzählt immer noch ihr...
...,...
202396,"@JoSteiniger Diese blöden Bürger, diese blöden..."
202397,@erik_donner @JoSteiniger Aber natürlich doch....
202398,@JoSteiniger @jansweber Gegen Olympia wurde be...
202399,"@JoSteiniger zulassen, dass korrupte Funktionä..."


In [11]:
swear_list = swear_df.text.to_list()

In [21]:
swear_list[:5]

['1/2\nIhr #Verräter\nIhr #Eidbrecher\nIhr #Verbrecher\n\nVerteiler! \n\n@cdu_fraktion @csu @Die_Gruenen\n@dieLinke @jensspahn \n@AfDimBundestag \n@fdp @HeikoMaas @GregorGysi @SawsanChebli @_FriedrichMerz\n@RegSprecher @akk @ABaerbock\n@DietmarBartsch @MarcoBuschmann \n\nhttps://t.co/OVKeQRvSDp',
 '@SWagenknecht Weil Sie das ja auch so gut wissen?!\nSo, wie Sie wussten, dass #Putin ja niiieeemals die #Ukraine überfällt.\nOder wie Sie sich mit #Corona auskennen und den ganzen #Querdenker und #Schwurbler Mist verbreiten.😒🙄🙈',
 '@ABW67484335 @susi_lins @SWagenknecht Ach wie schön: Persönliche Anekdötchen vs. Wissenschaft. So kennen wir unsere Querdullis. Ach so: Sind Ampeln auch Mist, weil Sie es schon mal vor einem heranrasenden Wagen über die Straße geschafft haben?',
 '@McI0vin @Bauernopfer1 @voglerk Ich weiß ja, ihr drei haltet euch für besonders schlau und lustig... aber dann würde ich vielleicht wenigstens die ganze transphobe Scheiße im Profil vorher verstecken, bevor ich so eine g

In [15]:
# Import germansentiment model: https://huggingface.co/oliverguhr/german-sentiment-bert
from germansentiment import SentimentModel

model = SentimentModel()

In [38]:
text = swear_list[5]

In [39]:
# Example:
classes, probabilities = model.predict_sentiment([text], output_probabilities = True) 
print(classes, probabilities)

['negative'] [[['positive', 0.059055864810943604], ['negative', 0.9357224702835083], ['neutral', 0.005221561994403601]]]


In [40]:
# Testing the model and display the perdiction score
for i in range(3):  
    if probabilities[0][i][0] == classes[0]:
        if classes[0] == "negative":
            if probabilities[0][i][1] > 0.6:
                print(f"The prediction is {probabilities[0][i][0]} with a Score of {probabilities[0][i][1]}")

The prediction is negative with a Score of 0.9357224702835083


In [48]:
# Append model to all tweets (from the Lexicon-based approach):

f = IntProgress(min=0, max=len(swear_list)) # instantiate the bar
display(f) # display the bar
print("PROGRESS")

sentiment_list = []
counter = 0

for i in swear_list:
    f.value += 1 # increment the progress bar
    counter += 1
    classes, probabilities = model.predict_sentiment([i], output_probabilities = True)
    for j in range(3):  
        if probabilities[0][j][0] == classes[0]:
            if classes[0] == "negative":
                if probabilities[0][j][1] > 0.9:
                    sentiment_list.append(i)
                    #print(f"The prediction is {probabilities[0][j][0]} with a Score of {probabilities[0][j][1]}")

len(sentiment_list)

IntProgress(value=0, max=202401)

PROGRESS


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



86719

In [50]:
# save resulting dataframe
df = pd.DataFrame(data={"text": sentiment_list})
df = df.reset_index()
df.to_csv("./sentiment_swear_words.csv", sep=';',index=False, encoding="utf-8")

In [52]:
# Sample 2000 Tweets that may be used for the Annotation Task
annotation = df.sample(2000)
annotation = annotation.reset_index()
annotation.to_csv("sentiment_annotation_2k.csv", encoding = "utf-8", sep=";")